# Python and C

---

Simple example for combining python and C code together. To build the executable, `cmake` is used. Let's create a `c` file.

In [15]:
%%file hello.c

#include <stdio.h>
#include <math.h>

int hello(int j);

int hello(int j){
    int i;
    int sum = 0;
    for(i=0; i<j; ++i){
        sum += cos(0.1);
        // printf("hello world: %d\n", sum);
    }
    
    return sum;
}

Overwriting hello.c


Now let's use `cmake` to help us compile our program ... `MakeFiles` suck!

In [16]:
%%file CMakeLists.txt

cmake_minimum_required (VERSION 3.7)
project(HELLO)
set(CMAKE_BUILD_TYPE Release)
set(CMAKE_LIBRARY_OUTPUT_DIRECTORY ${CMAKE_SOURCE_DIR}/lib)

IF( WIN32 )
    MESSAGE("Windows not supported ... ha, ha, ha, your OS sucks!")
ELSE( WIN32 )
    add_library(hello SHARED hello.c)
ENDIF( WIN32 )

Overwriting CMakeLists.txt


Now, we want to separate our code and build artifacts so let's create a `build` directory. This keeps everything nice looking. It also allows us to delete `build` and start over from a clean state without having to worry about acidentally deleting something 

In [17]:
!mkdir build

In [18]:
!cd build && cmake ..

-- The C compiler identification is AppleClang 8.0.0.8000042
-- The CXX compiler identification is AppleClang 8.0.0.8000042
-- Check for working C compiler: /Library/Developer/CommandLineTools/usr/bin/cc
-- Check for working C compiler: /Library/Developer/CommandLineTools/usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /Library/Developer/CommandLineTools/usr/bin/c++
-- Check for working CXX compiler: /Library/Developer/CommandLineTools/usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Configuring done
-- Generating done
-- Build files have been written to: /Users/kevin/github/robotics/python_c/build


In [19]:
!cd build && make

Scanning dependencies of target hello
[ 50%] Building C object CMakeFiles/hello.dir/hello.c.o
[100%] Linking C shared library ../lib/libhello.dylib
[100%] Built target hello


In [20]:
!file ./lib/libhello.dylib

./lib/libhello.dylib: Mach-O 64-bit dynamically linked shared library x86_64


In [14]:
%%file hello.py

import numpy
import ctypes

_libhello = numpy.ctypeslib.load_library('libhello', './lib')

_libhello.hello.argtypes = [ctypes.c_int]
_libhello.hello.restype  =  ctypes.c_int

def hello(n):
    return _libhello.hello(int(n))

Overwriting hello.py


In [5]:
import hello

In [6]:
hello.hello(1000)

0

In [ ]:
!ls

In [7]:
timeit hello.hello(1000)

The slowest run took 13.16 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 7.06 µs per loop


In [ ]:
# Let's make a pure python example for comparison
from math import cos

def hellotest(n):
    sum = 0
    for i in range(n):
        sum += cos(0.1)
    return sum

In [ ]:
timeit hellotest(1000)

http://stackoverflow.com/questions/24071491/how-can-i-make-a-python-wheel-from-an-existing-native-library


project:

    MD5
    │   setup.py
    │
    └───md5
        __init__.py   
        libeay32.dll
        
`setup.py`:

    from setuptools import setup, Distribution


    class BinaryDistribution(Distribution):
        def has_ext_modules(foo):
            return True


    setup(
        name='md5',
        version='1.0',
        description='MD5 Library',
        packages=['md5'],
        package_data={
            'md5': ['libeay32.dll'],
        },
        distclass=BinaryDistribution
    )

`__init__.py`:

    lib_path = os.path.join(os.path.dirname(__file__), 'libeay32.dll')
    lib = CDLL(lib_path)